In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
df = pd.read_csv(r"C:\Users\Dwij patel\email_dataset\spam_ham_dataset.csv")
labels = df['label']
labels

0        ham
1        ham
2        ham
3       spam
4        ham
        ... 
5166     ham
5167     ham
5168     ham
5169     ham
5170    spam
Name: label, Length: 5171, dtype: object

In [3]:
df.columns

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')

In [4]:
df = df.drop('Unnamed: 0',axis=1)
df = df.drop('label_num',axis=1)
df.head()

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...


In [5]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
class DfSelector(BaseEstimator,TransformerMixin):
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        X['label'] = (X['label']=='spam')
        X['label'] = X['label'].replace({True:1,False:0})
        return X
    

class TextTransform(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        X['text'] = X['text'].str.lower()
        lis = X['text'].tolist()
        label_list = X['label'].tolist()
        #print(label_list)
        
        count_vec = CountVectorizer(max_features = 23000)
        count_matrix = count_vec.fit_transform(lis)
        matr = count_matrix.toarray().astype(np.int8)

        count_col = count_vec.get_feature_names_out()
        data = pd.DataFrame(data=matr,columns=count_col)
        data['label'] = label_list
        
        return data
        

In [7]:
from sklearn.pipeline import Pipeline

pipe1 = Pipeline([
    ('df_sel',DfSelector()),
    ('text_vector',TextTransform())
])

In [8]:
new_df = pipe1.fit_transform(df)

X = new_df.drop('label',axis=1)
y = new_df['label']
y

0       0
1       0
2       0
3       1
4       0
       ..
5166    0
5167    0
5168    0
5169    0
5170    1
Name: label, Length: 5171, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

model_sgd = SGDClassifier()
sgd_params = {'alpha':[0.0001,0.001,0.01,0.1],'max_iter':[100,500,1000]}
grid_sgd = GridSearchCV(model_sgd,sgd_params)


model_svc = SVC()
svc_params = {'C':[1,3,5,10],'kernel':['Linear','poly'],'degree':[1,2,3]}
grid_svc = GridSearchCV(model_svc,svc_params)

model_nb = GaussianNB()



In [11]:
# train all three models

grid_sgd.fit(X_train,y_train)

GridSearchCV(estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'max_iter': [100, 500, 1000]})

In [ ]:
grid_svc.fit(X_train,y_train)

In [ ]:
model_nb.fit(X_train,y_train)

In [ ]:
grid_sgd.best_params_
best_sgd = SGDClassifier(alpha=grid_sgd.best_params_['alpha'],max_iter=grid_sgd.best_params_['max_iter'])

In [ ]:
# function to display cross_val_score of model
from slearn.model_selection import cross_val_score

def display_cross_val_score(estimator,X_train,y_train,cv):
    accuracy = cross_val_score(estimator,X_train,y_train,cv=cv)
    return accuracy

In [ ]:
# funtion to return confusion matrix of given y_pred and y_train
from sklearn.metrics import confusion_matrix

def display_confu_matrix(estimator,X_train,y_train):
    y_pred = estimator.predict(X_train)
    confu = confustion_matrix(y_train,y_pred)
    return confu
